In [0]:
from config import Config
from orchestration.state_manager import StateManager
from importlib import reload

import agents.parser_agent
reload(agents.parser_agent)

from agents.parser_agent import ParserAgent

from utils import delta_helpers

In [0]:
config = Config()
setup_result = config.setup_all(spark)
display(setup_result)

unity = config.unity
tables = config.tables

print("Volume root:", unity.volume_path)

# Funzione ricorsiva per raccogliere tutti i file .sas nel volume
all_sas_files = []

def collect_sas_files(base_path: str):
    entries = dbutils.fs.ls(base_path)
    for e in entries:
        if e.isDir():
            collect_sas_files(e.path)  # ricorsione su sottocartella
        else:
            if e.path.lower().endswith(".sas"):
                all_sas_files.append(e.path)

collect_sas_files(unity.volume_path)

print(f"Trovati {len(all_sas_files)} file .sas")
for p in all_sas_files[:10]:
    print(" -", p)

In [0]:
state_manager = StateManager(spark, config)

workflow_id = "manual_parse_all_001"
state_manager.create_workflow(workflow_id, initial_status="CREATED")

parser_agent = ParserAgent(
    config=config,
    spark=spark,
    state_manager=state_manager,
    workflow_id=workflow_id,
)

print("ParserAgent pronto, workflow_id:", workflow_id)

result = parser_agent.parse_files(all_sas_files)
print("Risultato parse_files:", result)